In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors

In [9]:
class glda :

    #--------------------- number of y that are 1 and 0------------------
    def get_len_class1(self,Y):
        self.num_y_1 = np.sum(Y)
        return self.num_y_1

    def get_len_class0(self,Y):
        self.num_y_0 = len(Y) - self.num_y_1
        return self.num_y_0

    #-------------------Calculate phi-------------------------------------
    def get_phi(self,Y):
        phi = self.num_y_1 / len(Y)
        return phi

    #-------------------Calculate mean for class 0(mu0)-------------------------------------
    def get_mu0(self,X,Y):
        num_features = X.shape[1]
        mu0 = np.zeros([num_features, ])
        for x, y in zip(X, Y):
            mu0 = mu0 + x * (1 if y == 0 else 0)
        mu0 = mu0 / self.num_y_0
        return mu0

    #-------------------Calculate mean for class 1(mu1)-------------------------------------
    def get_mu1(self,X,Y):
        num_features = X.shape[1]
        mu1 = np.zeros([num_features, ])
        for x, y in zip(X, Y):
            mu1 = mu1 + x * (1 if y == 1 else 0)
        mu1 = mu1 / self.num_y_1
        return mu1

    #-------------------Covariance Matrix-------------------------------------

    def get_covariance(self,X,Y,mu0,mu1):

        num_features = X.shape[1]
        mu0.shape = [num_features, 1]
        mu1.shape = [num_features, 1]
        sigma = np.zeros([num_features, num_features])

        for x, y in zip(X, Y):

            mu = mu0 if y == 0 else mu1
            x.shape = [num_features, 1]
            sigma = sigma + (x - mu) @ (x - mu).T

        sigma = sigma / (self.num_y_0 + self.num_y_1)
        return sigma


    #-----------------------decision boundary------------------------------
    def decision_boundary(self,x, mu0, mu1, sigma, phi):
        term1 = np.float64(((x).T @ np.linalg.inv(sigma) @ (mu0 - mu1)))
        term2 = np.float64(((mu1).T @ np.linalg.inv(sigma) @ (mu1)) / 2)
        term3 = np.float64(((mu0).T @ np.linalg.inv(sigma) @ (mu0)) / 2)
        term4 = np.log(phi / (1 - phi))
        return term1 + term2 - term3 + term4


    #-----------------------Plot the decision boundary------------------------------
    def plot_decision_boundary(self,x,mu0, mu1, sigma , phi ,plot):
        mu0 = mu0.reshape(2, 1)
        mu1 = mu1.reshape(2, 1)
        f0 = np.linspace(x[:,0].min(), x[:,0].max())
        f1 = np.linspace(x[:,1].min(), x[:,1].max())
        x1, x2 = np.meshgrid(f0, f1)
        z = np.zeros(x1.shape)
        sigma = np.matrix(sigma)
        for i in range(0, len(z)):
            for j in range(0, len(z[0])):
                x = np.array([x1[i][j], x2[i][j]]).reshape(2, 1)
                z[i][j] = self.decision_boundary(x, mu0, mu1, sigma, phi)
        db = plot.contour(x1, x2, z,levels=[0], colors='black')
        #db = plot.contourf(x1, x2, z,1,alpha=.5)
        return db

    #--------------------------predict dataset------------------------------------
    def predict(self,x,Mu0,Mu1,Sigma,Phi):
        preds = []
        Mu0 = np.array(Mu0).reshape(1,2)
        Mu1 = np.array(Mu1).reshape(1,2)
        Mu = np.concatenate((Mu0,Mu1))
        for i in range(x.shape[0]):
            preds.append(self.predict_sample(x[i],Mu,Sigma,Phi))
        return np.array(preds)
    #---------------------------predict a data-------------------------------------
    def predict_sample(self,x,Mu,Sigma,Phi):
        labels = [0,1]
        phi = [1-Phi,Phi]
        max_label = 0
        max_likelihood = 0
        const = 1./((2*np.pi)*np.linalg.det(Sigma)**(0.5))
        for i in range(len(labels)):
            likelihood  = const* np.exp(-1/2 * (x - Mu[i]).T @ np.linalg.inv(Sigma) @ (x - Mu[i]) ) * phi[i]
            #likelihood  = np.exp(-1/2 * (x - Mu[i]).T @ np.linalg.inv(Sigma) @ (x - Mu[i]) ) * phi[i]
            if likelihood > max_likelihood:
                max_label = labels[i]
                max_likelihood = likelihood
        return max_label

    #--------------------------accuracy------------------------------------
    def score(self,preds,y):
        correct0 = 0
        correct1 = 0
        num_y1 = np.sum(y)
        num_y0 = len(y) - num_y1
        score = (preds == y).mean()
        for i in range(len(y)):
            if (preds[i] == y[i] == 0):
                correct0 = correct0 + 1
            if (preds[i] == y[i] == 1):
                correct1 = correct1 + 1
        print("class0: toatal sample=%d | correct = %d" %(num_y0,correct0))
        print("class1: toatal sample=%d | correct = %d" %(num_y0,correct1))
        return score


    #---------------------------plot--------------------------------------------
    def plot2d(self,x,Mu0,Mu1,sigma,Phi):
        fig2 = plt.figure()
        ax2 = fig2.add_subplot(1,1,1)
        ax2.set_xlabel("x1")
        ax2.set_ylabel("x2")
        fig2.suptitle('2D estimated PDFs and decision boundary')

        Mu0 = np.array(Mu0).reshape(1,2)
        Mu1 = np.array(Mu1).reshape(1,2)
        Mu = np.concatenate((Mu0,Mu1))

        f0 = np.linspace(x[:,0].min(),x[:,0].max())
        f1 = np.linspace(x[:,1].min(),x[:,1].max())

        for c in range(2):
            const = 0
            const = 1./((2*np.pi)*np.linalg.det(sigma)**(0.5))
            X, Y = np.meshgrid(f0,f1)
            def pdf(point):
                return const*np.exp(-1/2.*(point-Mu[c]).T @ np.linalg.inv(sigma) @ (point-Mu[c]))
            z = np.array([pdf(np.array(ponit)) for ponit in zip(np.ravel(X),np.ravel(Y))])
            Z = z.reshape(X.shape)

            ax2.contour(X, Y, Z,linewidths=1)
            linboundary = self.plot_decision_boundary(x , Mu0, Mu1, sigma, Phi,ax2)



    def plot3d(self,x,Mu0,Mu1,sigma,Phi):
        fig3 = plt.figure(figsize=(8,8))
        ax3D = fig3.add_subplot(111, projection='3d')
        #ax3D = fig3.gca(projection='3d')
        fig3.suptitle('3D estimated PDF')
        ax3D.set_xlabel("x1")
        ax3D.set_ylabel("x2")
        ax3D.set_zlabel("P(X|y)")
        #ax3D.view_init(30,30)

        Mu0 = np.array(Mu0).reshape(1,2)
        Mu1 = np.array(Mu1).reshape(1,2)
        Mu = np.concatenate((Mu0,Mu1))

        f0 = np.linspace(x[:,0].min(), x[:,0].max())
        f1 = np.linspace(x[:,1].min(), x[:,1].max())
        X, Y = np.meshgrid(f0,f1)

        for c in range(2):
            const = 0
            const = 1./((2*np.pi)*np.linalg.det(sigma)**(0.5))
            def pdf(point):
                return const*np.exp(-1/2.*(point-Mu[c]).T @ np.linalg.inv(sigma) @ (point-Mu[c]))
            zs = np.array([pdf(np.array(ponit)) for ponit in zip(np.ravel(X),np.ravel(Y))])
            Z = zs.reshape(X.shape)

            ax3D.plot_surface(X, Y,Z,alpha=.3,cmap='ocean')
            ax3D.contour3D(X, Y,Z,60,cmap='ocean')
            print('\n')

